In [7]:
# read in imports
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score

In [2]:
# set up folder link and files
folder_link = 'C:/Users/Michael/projects/kaggle/rossman/'

train_file = folder_link + 'train.csv'
test_file = folder_link + 'test.csv'
submission_file = folder_link + 'predictions_DecisionTrees.csv'
store_file = folder_link + 'store.csv'

In [3]:
# set up dataframes
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
store = pd.read_csv(store_file)

C:\Users\Michael\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
# merge train and store data
train1 = train

# change state holiday to numeric
train1.StateHoliday = train1.StateHoliday.map({0:0,'0':1,'a':2,'b':3,'c':4})
train1.StateHoliday = train1.StateHoliday.astype(int)

In [5]:
# create features and response
feature_cols = ['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday']
train2 = train1.sample(n=100000)
X = train2[feature_cols]
y = train2.Sales

In [9]:
# set up decision tree with max depth of 1
treereg = DecisionTreeRegressor(max_depth=1,random_state=1)
scores = cross_val_score(treereg, X, y, cv=3, scoring='mean_squared_error')
np.mean(np.sqrt(-scores))

2828.4431600945568

In [10]:
# loop through many max depth values
max_depth_range = range(1,11)

# create an empty list to store RMSE
RMSE_scores = []

# use cross-validation with each value of max depth
for depth in max_depth_range:
    treereg = DecisionTreeRegressor(max_depth=depth,random_state=1)
    MSE_scores = cross_val_score(treereg, X, y, cv=3, scoring='mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

# print results
RMSE_scores


[2828.4431600945568,
 2630.2837414418022,
 2580.8118306689357,
 2576.7990370641965,
 2572.6010225926152,
 2567.3072125924305,
 2565.5268714822582,
 2565.0522527035459,
 2564.7764416668274,
 2564.867703419693]

In [12]:
# create decision tree based on max depth of 3
treereg = DecisionTreeRegressor(max_depth=3,random_state=1)
treereg.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=1, splitter='best')

In [21]:
# remove NANs from test
test.loc[test.Open.isnull(),'Open'] = 1
test.loc[test.StateHoliday.isnull(),'StateHoliday'] = 0
test.StateHoliday = test.StateHoliday.map({0:0,'0':1,'a':2,'b':3,'c':4})
test.StateHoliday = test.StateHoliday.astype(int)

In [1]:
# predict values using decision tree
y_pred = treereg.predict(test[feature_cols])
y_pred

NameError: name 'treereg' is not defined

In [23]:
# create submission dataframe
submit = pd.DataFrame({'Id':test['Id'],'Sales':y_pred})

In [24]:
# create csv file
submit.to_csv(submission_file,index=False)